<a href="https://colab.research.google.com/github/MaksimNikolin/Carservice/blob/main/test_model_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import libraries
!pip install onnx
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.6 MB/s eta 0:00:00


In [2]:
#import functions
import os
import time
import datetime
import cv2
import numpy as np
import tensorflow as tf
import xml.etree.ElementTree as ET


def seconds_to_m_s(seconds):
    m = int(seconds // 60)
    s = int(seconds % 60)
    return m, s


def preprocess_input(img, input_w, input_h):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    image = img.copy()
    image = cv2.resize(image, (input_w, input_h))
    image = image.transpose((2, 0, 1))
    image = np.expand_dims(image, 0)
    image = np.ascontiguousarray(image)
    im = image.astype(np.float32)
    im /= 255
    return im


def detect_all(image, sessions, innames, outnames, inputs_w, inputs_h, SCORE_THRESH, shift_classes):
    bboxes = []
    for idx, (session, inname, outname, input_w, input_h, thresh) in enumerate(zip(sessions,
                                                                               innames,
                                                                               outnames,
                                                                               inputs_w,
                                                                               inputs_h,
                                                                               SCORE_THRESH)):
        im = preprocess_input(image, input_w, input_h)
        inp = {inname[0]: im}
        outputs = session.run(outname, inp)[0]
        for i, (batch_id, x0, y0, x1, y1, cls_id, score) in enumerate(outputs):
            if score > thresh:
                x0 = int(x0 / input_w * image.shape[1])
                y0 = int(y0 / input_h * image.shape[0])
                x1 = int(x1 / input_w * image.shape[1])
                y1 = int(y1 / input_h * image.shape[0])
                bboxes.append([shift_classes[idx] + int(cls_id), x0, y0, x1, y1])
    return bboxes


def prepare_classes(classes):
    start_pos = [0] * len(classes)
    class_names = classes[0]
    for i in range(1, len(classes)):
        start_pos[i] = start_pos[i-1] + len(classes[i-1])
        class_names += classes[i]
    return start_pos, class_names


def get_all_classes(boxes, start_pos):
    for i in range(len(boxes)):
        model_idx, label = boxes[i][4]
        class_id = start_pos[model_idx] + label
        boxes[i][4] = class_id
    return boxes


def detect(image, session, inname, outname, input_w, input_h, SCORE_THRESH):
    bboxes = []
    labels = []
    im = preprocess_input(image, input_w, input_h)
    inp = {inname[0]: im}
    outputs = session.run(outname, inp)[0]
    for i, (batch_id, x0, y0, x1, y1, cls_id, score) in enumerate(outputs):
        if score > SCORE_THRESH:
            x0 = int(x0 / input_w * image.shape[1])
            y0 = int(y0 / input_h * image.shape[0])
            x1 = int(x1 / input_w * image.shape[1])
            y1 = int(y1 / input_h * image.shape[0])
            bboxes.append([int(cls_id), x0, y0, x1, y1])
    return bboxes


def create_xml_file(folder, path, file_name, all_boxes):
    return f'''<?xml version="1.0" ?>
<annotation>
    <folder>{folder}</folder>
    <filename>{file_name}</filename>
    <path>{path}</path>
    <source>
        <database>Unknown</database>
    </source>
    <segmented>0</segmented>
    {all_boxes}
</annotation>'''


def create_xml_box(class_name, box):
    xmin,ymin,xmax,ymax = box
    box_form = f'''<object>
    <name>{class_name}</name>
    <pose>Unspecified</pose>
    <truncated>0</truncated>
    <difficult>0</difficult>
    <bndbox>
        <xmin>{xmin}</xmin>
        <ymin>{ymin}</ymin>
        <xmax>{xmax}</xmax>
        <ymax>{ymax}</ymax>
    </bndbox>
</object>'''
    return box_form


def get_boxes_xml(image, bboxes, class_names, use_xml=True, draw_box=True):
    points = []
    all_boxes_form = ''
    for i, box in enumerate(bboxes):
        points.append(box)
        xmin = int(max(box[1],0))
        ymin = int(max(box[2],0))
        xmax = int(min(box[3],image.shape[1]))
        ymax = int(min(box[4], image.shape[0]))
        class_id = int(box[0])
        box = (xmin, ymin, xmax, ymax)
        if use_xml:
            box_form = create_xml_box(class_names[class_id], box)
            all_boxes_form += box_form
        if draw_box:
            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
    return all_boxes_form, points


def xml_to_boxes(xml_file):
    tree = ET.parse(open(xml_file))
    root = tree.getroot()
    boxes = []
    for object in root.iter('object'):
        box_form = object.find('bndbox')
        xmin = int(box_form.find('xmin').text)
        ymin = int(box_form.find('ymin').text)
        xmax = int(box_form.find('xmax').text)
        ymax = int(box_form.find('ymax').text)
        boxes.append((xmin, ymin, xmax, ymax))
    return boxes


def start_processing_time():
    start_time_hms = datetime.datetime.now()
    print('Work started in {}:{:02d}:{:02d}'.format(start_time_hms.hour, start_time_hms.minute, start_time_hms.second))
    return time.time()


def calculated_left_time(total_save, start_pocessing, frame_ind, serial_number, frame_ind_load):
    pass_images = int((frame_ind - frame_ind_load) / serial_number)
    left_images = total_save - int(frame_ind / serial_number)
    if left_images > 0:
        left_time_sec = int((time.time() - start_pocessing) * left_images / pass_images)
        print('It remains to process the images:', left_images)
        left_time = datetime.timedelta(seconds=left_time_sec)
        print('It will take:', str(left_time))
        finish_time = datetime.datetime.now() + left_time
        print('Approximately processing will end in {}:{:02d}:{:02d}'.format(finish_time.hour, finish_time.minute, finish_time.second))

In [3]:
import glob
import os
import cv2
import time
import pickle
import random
import shutil
import onnxruntime as ort
import glob
import numpy as np
from string import ascii_lowercase
#from tools.utils import (detect, seconds_to_m_s, get_boxes_xml, create_xml_file, start_processing_time, calculated_left_time)
from tqdm import tqdm

*Create folder named 'Model_data' and put our model 'best.onnx' in it.
Create folder named 'u1' and put unmarked images in it.*

In [5]:
start_work = time.time()

frames_dir = 'u1' #path to the folder with input frames
models_dir = 'Model_data' #folder for models
model_name = 'best.onnx' #model name

model_input_size = (288,512)
input_h = model_input_size[0]
input_w = model_input_size[1]

cuda = False
providers = ['CUDAExecutionProvider', 'CPUExecutionProvider'] if cuda else ['CPUExecutionProvider']

class_names = ['car','person','plate'] #label names
SCORE_THRESH = 0.5 #the threshold of probability below which the prediction is filtered out
checkpoint_size = 100 #how many frames must pass before saving the checkpoint, taking into account the skipping of frames
USE_XML = True #use as output labels: False - use only the label dictionary, True - use additional xml tags
DRAW_BOXES = True #do I need to create test images in folder named 'Draw' with the head frames drawn


model_path = os.path.join(models_dir, model_name)
session = ort.InferenceSession(model_path, providers=providers)
inname = [i.name for i in session.get_inputs()]
outname = [i.name for i in session.get_outputs()]


if DRAW_BOXES:
    draw_dir = os.path.join(frames_dir, 'Draw')
    if not os.path.exists(draw_dir):
        os.mkdir(draw_dir)


processing_start = start_processing_time()
print('\nFrames are being processed and saved...')
all_imgs = glob.glob(frames_dir + '/*.jpeg')
count_save = 0
for fil in tqdm(all_imgs):
    image = cv2.imread(fil)
    image_name = os.path.basename(fil)
    image_name_short = image_name.split('.')[0]
    bboxes = np.array(detect(image, session, inname, outname, input_w, input_h, SCORE_THRESH))
    if len(bboxes) != 0:
        bboxes = bboxes[bboxes[:, -1] > 250]
        all_boxes_form, points = get_boxes_xml(image, bboxes, class_names, USE_XML, DRAW_BOXES)
        if DRAW_BOXES:
            cv2.imwrite(os.path.join(draw_dir, image_name), image)
        if USE_XML:
            form_xml = create_xml_file(frames_dir, frames_dir, image_name, all_boxes_form)
            with open(os.path.join(frames_dir, image_name_short + '.xml'), 'w') as file:
                file.write(form_xml)
        count_save += 1

    # if count_save % checkpoint_size == 0:
    #     print('\nProcessed frames:', count_save)
    #     calculated_left_time(len(all_imgs), processing_start, count_save, 1, 0)

print('Frame saving completed.')

duration = time.time()-start_work
processing_duration = time.time()-processing_start
print('\nTotal video processing took {} minuts and {} seconds'.format(*seconds_to_m_s(duration)))
print('More accuratelly:', round(duration, 2))
print('Processing only:', round(processing_duration, 2))

Work started in 13:43:06

Frames are being processed and saved...


100%|██████████| 4/4 [00:00<00:00,  5.75it/s]

Frame saving completed.

Total video processing took 0 minuts and 0 seconds
More accuratelly: 0.86
Processing only: 0.71
